In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import *

from overrides import overrides
import warnings

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util


DATA_ROOT = Path("../data/jigsaw")

In [18]:
label_cols = ["toxic", "severe_toxic", "obscene",
              "threat", "insult", "identity_hate"]

In [19]:
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.fields import TextField, SequenceLabelField, LabelField, MetadataField, ArrayField

class JigsawDatasetReader(DatasetReader):
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None, # TODO: Handle mapping from BERT
                 max_seq_len: Optional[int]=None) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    @overrides
    def text_to_instance(self, tokens: List[Token], id: str,
                         labels: np.ndarray) -> Instance:
        sentence_field = TextField([Token(x) for x in tokens],
                                   self.token_indexers)
        fields = {"tokens": sentence_field}
        
        id_field = MetadataField(id)
        fields["id"] = id_field
        
        meta_field = MetadataField({"lengths": np.array([len(t) for t in tokens])})
        fields["meta"] = meta_field
        
        label_field = ArrayField(array=labels)
        fields["label"] = label_field

        return Instance(fields)
    
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path)
        for i, row in df.iterrows():
            yield self.text_to_instance(
                self.tokenizer(row["comment_text"]),
                row["id"], row[label_cols].values,
            )

In [20]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.data.token_indexers import WordpieceIndexer, SingleIdTokenIndexer

_spacy_tok = SpacyWordSplitter(language='en_core_web_sm', pos_tags=False).split_words

from allennlp.data.token_indexers import SingleIdTokenIndexer
token_indexer = SingleIdTokenIndexer(
    lowercase_tokens=True,
)
def tokenizer(x: str):
    return [w.text for w in
            _spacy_tok(x.lower())]

In [21]:
reader = JigsawDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [22]:
train_ds, test_ds = (reader.read(DATA_ROOT / fname) for fname in ["train.csv",
                                                                  "test_proced.csv"])



0it [00:00, ?it/s]

1it [00:02,  2.38s/it]

9it [00:02,  1.67s/it]

16it [00:02,  1.17s/it]

23it [00:02,  1.21it/s]

30it [00:02,  1.72it/s]

36it [00:02,  2.42it/s]
42it [00:03,  3.23it/s]
47it [00:03,  4.47it/s]
52it [00:03,  6.05it/s]
57it [00:03,  8.20it/s]
62it [00:03, 10.12it/s]
70it [00:04, 13.66it/s]
77it [00:04, 17.97it/s]
83it [00:04, 22.07it/s]
92it [00:04, 28.30it/s]
99it [00:04, 34.17it/s]
106it [00:04, 39.25it/s]
113it [00:04, 39.56it/s]
119it [00:04, 42.47it/s]
127it [00:05, 43.52it/s]
135it [00:05, 50.08it/s]
143it [00:05, 53.66it/s]
150it [00:05, 55.85it/s]
157it [00:05, 51.71it/s]
163it [00:05, 41.73it/s]
171it [00:05, 48.67it/s]
181it [00:05, 56.02it/s]
188it [00:06, 57.52it/s]
195it [00:06, 52.04it/s]
201it [00:06, 46.20it/s]
207it [00:06, 42.88it/s]
212it [00:06, 43.64it/s]
217it [00:06, 40.48it/s]
226it [00:06, 47.54it/s]
232it [00:07, 49.87it/s]
238it [00:07, 51.57it/s]
244it [00:07, 46.07it/s]
250it [00:07, 48.35it/s]
256it [00:07, 47.98it/s]
261it [00:07, 40

2377it [00:40, 95.89it/s]
2402it [00:40, 117.50it/s]
2417it [00:40, 115.94it/s]
2431it [00:40, 57.39it/s] 
2442it [00:40, 61.80it/s]
2452it [00:41, 69.57it/s]
2465it [00:41, 80.61it/s]
2476it [00:41, 84.90it/s]
2487it [00:41, 85.95it/s]
2499it [00:41, 92.18it/s]
2510it [00:41, 74.60it/s]
2519it [00:41, 77.74it/s]
2530it [00:41, 85.08it/s]
2567it [00:42, 110.58it/s]
2587it [00:42, 127.21it/s]
2610it [00:42, 145.74it/s]
2630it [00:42, 135.90it/s]
2648it [00:42, 125.81it/s]
2664it [00:42, 126.55it/s]
2679it [00:42, 121.26it/s]
2693it [00:42, 121.92it/s]
2707it [00:43, 116.75it/s]
2720it [00:43, 109.69it/s]
2733it [00:43, 113.68it/s]
2745it [00:43, 108.86it/s]
2757it [00:43, 105.80it/s]
2769it [00:43, 109.42it/s]
2781it [00:43, 110.19it/s]
2795it [00:43, 116.89it/s]
2807it [00:43, 115.67it/s]
2829it [00:44, 132.28it/s]
2864it [00:44, 162.26it/s]
2885it [00:44, 154.23it/s]
2904it [00:44, 152.51it/s]
2922it [00:44, 132.28it/s]
2939it [00:44, 140.88it/s]
2955it [00:44, 141.95it/s]
2971it [00:

7589it [01:14, 104.71it/s]
7603it [01:15, 108.58it/s]
7616it [01:15, 111.34it/s]
7629it [01:15, 113.51it/s]
7642it [01:15, 96.68it/s] 
7653it [01:15, 83.32it/s]
7663it [01:15, 86.68it/s]
7678it [01:15, 97.80it/s]
7693it [01:15, 108.91it/s]
7719it [01:16, 131.54it/s]
7753it [01:16, 160.35it/s]
7775it [01:16, 156.63it/s]
7795it [01:16, 128.19it/s]
7812it [01:16, 120.70it/s]
7827it [01:16, 123.08it/s]
7845it [01:16, 131.06it/s]
7860it [01:17, 130.04it/s]
7876it [01:17, 137.47it/s]
7891it [01:17, 99.04it/s] 
7903it [01:17, 94.41it/s]
7914it [01:17, 91.38it/s]
7925it [01:17, 88.62it/s]
7935it [01:17, 81.78it/s]
7957it [01:18, 81.44it/s]
7970it [01:18, 90.79it/s]
7980it [01:18, 62.16it/s]
7988it [01:18, 64.46it/s]
7998it [01:18, 71.85it/s]
8010it [01:18, 81.55it/s]
8025it [01:19, 93.01it/s]
8038it [01:19, 101.20it/s]
8050it [01:19, 83.01it/s] 
8060it [01:19, 81.76it/s]
8070it [01:19, 75.02it/s]
8079it [01:19, 76.99it/s]
8091it [01:19, 85.26it/s]
8101it [01:19, 83.92it/s]
8125it [01:20, 103.7

15363it [01:49, 291.84it/s]
15394it [01:49, 295.88it/s]
15429it [01:49, 308.94it/s]
15461it [01:49, 306.25it/s]
15503it [01:49, 332.66it/s]
15538it [01:49, 320.96it/s]
15571it [01:49, 286.12it/s]
15618it [01:49, 322.60it/s]
15662it [01:50, 350.64it/s]
15700it [01:50, 348.25it/s]
15737it [01:50, 332.05it/s]
15772it [01:50, 303.24it/s]
15804it [01:50, 282.60it/s]
15834it [01:50, 264.25it/s]
15862it [01:50, 239.00it/s]
15894it [01:50, 257.34it/s]
15927it [01:50, 275.12it/s]
15962it [01:51, 293.09it/s]
15997it [01:51, 304.97it/s]
16032it [01:51, 316.57it/s]
16069it [01:51, 329.94it/s]
16103it [01:51, 332.39it/s]
16137it [01:51, 317.58it/s]
16170it [01:51, 315.77it/s]
16205it [01:51, 324.90it/s]
16238it [01:51, 309.55it/s]
16279it [01:52, 332.48it/s]
16313it [01:52, 312.95it/s]
16346it [01:52, 303.22it/s]
16387it [01:52, 327.47it/s]
16421it [01:52, 319.62it/s]
16454it [01:52, 274.21it/s]
16484it [01:52, 239.62it/s]
16510it [01:52, 220.22it/s]
16538it [01:53, 228.74it/s]
16573it [01:53, 251.

24357it [02:19, 350.73it/s]
24393it [02:19, 348.48it/s]
24432it [02:19, 357.45it/s]
24471it [02:19, 366.18it/s]
24509it [02:19, 369.43it/s]
24547it [02:20, 358.00it/s]
24584it [02:20, 345.00it/s]
24619it [02:20, 341.65it/s]
24654it [02:20, 331.88it/s]
24688it [02:20, 331.52it/s]
24722it [02:20, 315.01it/s]
24754it [02:20, 284.26it/s]
24786it [02:20, 289.45it/s]
24816it [02:20, 289.32it/s]
24846it [02:21, 277.07it/s]
24882it [02:21, 297.42it/s]
24917it [02:21, 311.07it/s]
24949it [02:21, 311.69it/s]
24985it [02:21, 311.96it/s]
25017it [02:21, 309.15it/s]
25050it [02:21, 313.15it/s]
25092it [02:21, 331.14it/s]
25127it [02:21, 335.69it/s]
25161it [02:22, 324.49it/s]
25194it [02:22, 323.08it/s]
25227it [02:22, 312.17it/s]
25268it [02:22, 335.38it/s]
25307it [02:22, 348.83it/s]
25343it [02:22, 349.89it/s]
25379it [02:22, 304.44it/s]
25411it [02:22, 293.30it/s]
25451it [02:22, 317.91it/s]
25486it [02:23, 325.42it/s]
25521it [02:23, 331.86it/s]
25555it [02:23, 323.63it/s]
25596it [02:23, 343.

...

25649it [02:47, 268.79it/s]
25684it [02:47, 288.69it/s]
25716it [02:47, 295.47it/s]
25751it [02:47, 307.18it/s]
25783it [02:47, 291.58it/s]
25813it [02:47, 270.29it/s]
25843it [02:47, 253.56it/s]
25871it [02:47, 258.63it/s]
25905it [02:47, 252.86it/s]
25938it [02:48, 268.49it/s]
25967it [02:48, 274.11it/s]
25995it [02:48, 271.03it/s]
26023it [02:48, 257.52it/s]
26050it [02:48, 227.55it/s]
26074it [02:48, 217.84it/s]
26100it [02:48, 214.03it/s]
26145it [02:48, 253.98it/s]
26178it [02:49, 272.84it/s]
26209it [02:49, 277.38it/s]
26239it [02:49, 263.60it/s]
26267it [02:49, 260.43it/s]
26301it [02:49, 276.29it/s]
26330it [02:49, 261.09it/s]
26367it [02:49, 285.29it/s]
26403it [02:49, 303.82it/s]
26435it [02:49, 290.50it/s]
26466it [02:50, 267.32it/s]
26499it [02:50, 279.93it/s]
26528it [02:50, 279.59it/s]
26557it [02:50, 264.15it/s]
26592it [02:50, 284.37it/s]
26622it [02:50, 282.37it/s]
26651it [02:50, 250.51it/s]
26679it [02:50, 247.40it/s]
26713it [02:50, 266.10it/s]
26741it [02:51, 259.

33872it [03:17, 236.28it/s]
33897it [03:17, 232.08it/s]
33921it [03:17, 200.23it/s]
33948it [03:17, 215.17it/s]
33971it [03:18, 193.46it/s]
34000it [03:18, 213.54it/s]
34026it [03:18, 224.90it/s]
34061it [03:18, 233.96it/s]
34092it [03:18, 252.16it/s]
34131it [03:18, 280.30it/s]
34165it [03:18, 295.59it/s]
34199it [03:18, 303.97it/s]
34231it [03:18, 293.02it/s]
34262it [03:19, 254.99it/s]
34299it [03:19, 281.18it/s]
34329it [03:19, 266.86it/s]
34360it [03:19, 278.28it/s]
34389it [03:19, 255.28it/s]
34416it [03:19, 258.91it/s]
34446it [03:19, 269.08it/s]
34483it [03:19, 292.32it/s]
34514it [03:19, 271.91it/s]
34548it [03:20, 289.20it/s]
34582it [03:20, 301.63it/s]
34614it [03:20, 300.59it/s]
34645it [03:20, 299.93it/s]
34676it [03:20, 289.67it/s]
34708it [03:20, 297.92it/s]
34739it [03:20, 290.88it/s]
34772it [03:20, 300.48it/s]
34805it [03:20, 308.72it/s]
34837it [03:21, 299.26it/s]
34868it [03:21, 296.38it/s]
34901it [03:21, 294.04it/s]
34937it [03:21, 308.51it/s]
34969it [03:21, 300.

41997it [03:50, 253.81it/s]
42038it [03:50, 285.34it/s]
42070it [03:50, 258.67it/s]
42102it [03:50, 274.28it/s]
42136it [03:50, 288.90it/s]
42172it [03:50, 305.88it/s]
42204it [03:50, 285.54it/s]
42234it [03:50, 251.19it/s]
42261it [03:51, 207.80it/s]
42285it [03:51, 195.49it/s]
42307it [03:51, 180.04it/s]
42327it [03:51, 156.88it/s]
42350it [03:51, 169.97it/s]
42384it [03:51, 199.70it/s]
42420it [03:51, 228.77it/s]
42457it [03:52, 251.41it/s]
42488it [03:52, 260.33it/s]
42517it [03:52, 267.57it/s]
42549it [03:52, 279.31it/s]
42579it [03:52, 274.25it/s]
42608it [03:52, 275.83it/s]
42637it [03:52, 271.52it/s]
42665it [03:52, 260.28it/s]
42692it [03:53, 180.46it/s]
42714it [03:53, 137.34it/s]
42732it [03:53, 111.44it/s]
42747it [03:53, 103.36it/s]
42761it [03:53, 108.45it/s]
42783it [03:53, 127.54it/s]
42816it [03:54, 156.09it/s]
42849it [03:54, 185.40it/s]
42878it [03:54, 205.64it/s]
42912it [03:54, 232.77it/s]
42940it [03:54, 160.13it/s]
42963it [03:54, 119.12it/s]
42981it [03:55, 129.

49951it [04:25, 257.76it/s]
49987it [04:25, 280.84it/s]
50017it [04:25, 181.07it/s]
50055it [04:26, 214.42it/s]
50083it [04:26, 226.36it/s]
50121it [04:26, 255.22it/s]
50151it [04:26, 155.68it/s]
50175it [04:26, 154.91it/s]
50204it [04:26, 179.79it/s]
50231it [04:27, 199.10it/s]
50263it [04:27, 221.08it/s]
50289it [04:27, 116.21it/s]
50309it [04:27, 108.83it/s]
50326it [04:27, 100.83it/s]
50341it [04:28, 103.10it/s]
50355it [04:28, 93.46it/s] 
50377it [04:28, 110.78it/s]
50411it [04:28, 138.82it/s]
50447it [04:28, 169.75it/s]
50478it [04:28, 196.37it/s]
50505it [04:28, 153.87it/s]
50527it [04:29, 136.61it/s]
50546it [04:29, 138.68it/s]
50564it [04:29, 140.55it/s]
50581it [04:29, 142.73it/s]
50597it [04:29, 125.30it/s]
50614it [04:29, 133.59it/s]
50641it [04:29, 157.09it/s]
50660it [04:30, 163.96it/s]
50679it [04:30, 141.92it/s]
50695it [04:30, 134.19it/s]
50710it [04:30, 121.77it/s]
50724it [04:30, 101.94it/s]
50741it [04:30, 115.31it/s]
50763it [04:30, 133.20it/s]
50796it [04:30, 162.

57599it [05:03, 274.55it/s]
57630it [05:03, 280.93it/s]
57666it [05:03, 298.52it/s]
57697it [05:04, 281.70it/s]
57726it [05:04, 250.79it/s]
57753it [05:04, 223.46it/s]
57777it [05:04, 204.06it/s]
57800it [05:04, 208.03it/s]
57822it [05:04, 204.53it/s]
57844it [05:04, 178.14it/s]
57863it [05:05, 145.92it/s]
57880it [05:05, 150.17it/s]
57907it [05:05, 172.87it/s]
57945it [05:05, 206.53it/s]
57974it [05:05, 225.99it/s]
58006it [05:05, 244.91it/s]
58034it [05:05, 248.50it/s]
58068it [05:05, 269.92it/s]
58097it [05:05, 264.51it/s]
58126it [05:05, 270.59it/s]
58155it [05:06, 233.85it/s]
58180it [05:06, 157.65it/s]
58201it [05:06, 151.72it/s]
58220it [05:06, 160.61it/s]
58239it [05:06, 164.99it/s]
58281it [05:06, 200.54it/s]
58312it [05:06, 220.44it/s]
58338it [05:07, 191.39it/s]
58361it [05:07, 198.70it/s]
58394it [05:07, 224.88it/s]
58420it [05:07, 232.27it/s]
58456it [05:07, 244.08it/s]
58483it [05:07, 186.03it/s]
58505it [05:07, 165.38it/s]
58525it [05:08, 146.00it/s]
58542it [05:08, 129.

In [23]:
from allennlp.data.vocabulary import Vocabulary
vocab = Vocabulary.from_instances(train_ds + test_ds)

02/05/2019 15:17:06 - INFO - allennlp.data.vocabulary -   Fitting token dictionary from dataset.

 48%|████▊     | 108017/223549 [00:15<00:16, 7004.71it/s]


100%|██████████| 223549/223549 [00:30<00:00, 7216.49it/s] 

In [26]:
# vocab should have been saved using
vocab.save_to_files(DATA_ROOT / "vocab")

To avoid memory errors, restart here and build embedding matrix

In [3]:
from allennlp.data.vocabulary import Vocabulary
vocab = Vocabulary.from_files(DATA_ROOT / "vocab")

02/05/2019 15:26:24 - INFO - allennlp.data.vocabulary -   Loading token dictionary from ../data/jigsaw/vocab.


In [4]:
import fastText
ft_model = fastText.load_model(str(DATA_ROOT / "ft_model.bin"))

In [5]:
with (DATA_ROOT / "ft_model.txt").open("wt") as f:
    for idx, token in vocab.get_index_to_token_vocabulary().items():
        emb = ft_model.get_word_vector(token)
        emb_as_str = " ".join(["%.4f" % x for x in emb])
        f.write(f"{token} {emb_as_str}\n")